<a href="https://colab.research.google.com/github/adalbertii/azure-cognitive-service/blob/main/form_recognizer_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**OCR -owanie przykładowej faktury**
- wybranych pól
- wszystkich pól wraz z oreśleniem pewności identyfikacji poszczególnych pól

Wykorzystanie:
- modelu wytrenowanego na fakturach wyspecyfikowanych w standardzie US

- **API** - biblioteki **azure-ai-formrecognizer**

In [1]:
pip install azure-ai-formrecognizer==3.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [7]:
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

In [8]:
# parametry serwisu Azure FormRecognizer (form-reco-01)
#-----------------------------------------------------------------------------
endpoint = "https://form-reco-01.cognitiveservices.azure.com/"

key = "9e938ddce2f24a54ad0b95bd78bff72a"

In [9]:
# przykładow faktura
invoiceUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-invoice.pdf"

In [10]:
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def analyze_invoice():
# funkcja OCR-uje TYLKO wybrane pola z przykładowej faktury

    document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))

#----------------------------------------------------------------------------------------------------
# korzystamy z standardowego modelu (wyternwanego na standardowych dla US fakturach)
# --> "prebuilt-invoice"

    poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-invoice", invoiceUrl)
#----------------------------------------------------------------------------------------------------

    invoices = poller.result()
#----------------------------------------------------------------------------------------------------

    for idx, invoice in enumerate(invoices.documents):
        print("--------Rozpoznana faktura #{}--------".format(idx + 1))
        vendor_name = invoice.fields.get("VendorName")
        if vendor_name:
            print(
                "Nazwa dostawcy       : {}".format(
                    vendor_name.value
                )
            )
        vendor_address = invoice.fields.get("VendorAddress")
        if vendor_address:
            print(
                "Adres dostawcy       : {}".format(
                    vendor_address.value
                )
            )
        customer_name = invoice.fields.get("CustomerName")
        if customer_name:
            print(
                "Nazwa klienta        : {}".format(
                    customer_name.value
                )
            )


In [11]:
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def analyze_invoice_with_confidence():
# analogiczna do powyżej zdefiniowanej funkcja
# IDENTYFIKUJE WSZYSTKIE POLA FAKTURY
# wyświetla również poziom pewności klasyfikacji poszczególnych pól

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    poller = document_analysis_client.begin_analyze_document_from_url(
            "prebuilt-invoice", invoiceUrl)
    invoices = poller.result()

    for idx, invoice in enumerate(invoices.documents):
        print("--------Rozpoznana faktura #{}--------".format(idx + 1))
        vendor_name = invoice.fields.get("VendorName")
        if vendor_name:
            print(
                "Nazwa dostawcy       : {} pewność identyfikacji: {}".format(
                    vendor_name.value, vendor_name.confidence
                )
            )
        vendor_address = invoice.fields.get("VendorAddress")
        if vendor_address:
            print(
                "Adres dostawcy       : {} pewność identyfikacji: {}".format(
                    vendor_address.value, vendor_address.confidence
                )
            )
        customer_name = invoice.fields.get("CustomerName")
        if customer_name:
            print(
                "Nazwa klienta        : {} pewność identyfikacji: {}".format(
                    customer_name.value, customer_name.confidence
                )
            )
        invoice_id = invoice.fields.get("InvoiceId")
        if invoice_id:
            print(
                "Identyfikator faktury: {} pewność identyfikacji: {}".format(
                    invoice_id.value, invoice_id.confidence
                )
            )
        invoice_date = invoice.fields.get("InvoiceDate")
        if invoice_date:
            print(
                "Data faktury         : {} pewność identyfikacji: {}".format(
                    invoice_date.value, invoice_date.confidence
                )
            )
        invoice_total = invoice.fields.get("InvoiceTotal")
        if invoice_total:
            print(
                "Suma                 : {} pewność identyfikacji: {}".format(
                    invoice_total.value, invoice_total.confidence
                )
            )
        due_date = invoice.fields.get("DueDate")
        if due_date:
            print(
                "Termin płatnosci     : {} pewność identyfikacji: {}".format(
                    due_date.value, due_date.confidence
                )
            )
        purchase_order = invoice.fields.get("PurchaseOrder")
        if purchase_order:
            print(
                "Zamówienie           : {} pewność identyfikacji:{}".format(
                    purchase_order.value, purchase_order.confidence
                )
            )
        shipping_address = invoice.fields.get("ShippingAddress")
        if shipping_address:
            print(
                "Adres wysyłki        : {} pewność identyfikacji:{}".format(
                    shipping_address.value, shipping_address.confidence
                )
            )
        print("-----------------------------------------------")
        print("Pozycje na fakturze  :")
        for idx, item in enumerate(invoice.fields.get("Items").value):
            print("...Pozycja #{}".format(idx + 1))
            item_description = item.value.get("Description")
            if item_description:
                print(
                    "......Opis        : {} pewność identyfikacji: {}".format(
                        item_description.value, item_description.confidence
                    )
                )
            unit = item.value.get("Unit")
            if unit:
                print(
                    "......Jednostka   : {} pewność identyfikacji: {}".format(
                        unit.value, unit.confidence
                    )
                )
            unit_price = item.value.get("UnitPrice")
            if unit_price:
                print(
                    "......Cena jedn.  : {} pewność identyfikacji: {}".format(
                        unit_price.value, unit_price.confidence
                    )
                )
            product_code = item.value.get("ProductCode")
            if product_code:
                print(
                    "......KOd produktu: {} pewność identyfikacji:{}".format(
                        product_code.value, product_code.confidence
                    )
                )
            item_date = item.value.get("Date")
            if item_date:
                print(
                    "......Data       : {} pewność identyfikacji: {}".format(
                        item_date.value, item_date.confidence
                    )
                )
            tax = item.value.get("Tax")
            if tax:
                print(
                    "......Podatek   : {} pewność identyfikacji: {}".format(tax.value, tax.confidence)
                )
            amount = item.value.get("Amount")
            if amount:
                print(
                    "......Ilosć       : {} pewność identyfikacji: {}".format(
                        amount.value, amount.confidence
                    )
                )
        print("----------------------------------")
        subtotal = invoice.fields.get("SubTotal")
        if subtotal:
            print(
                "Suma częściowa    : {} pewność identyfikacji: {}".format(
                    subtotal.value, subtotal.confidence
                )
            )
        total_tax = invoice.fields.get("TotalTax")
        if total_tax:
            print(
                "POdatek całkowity : {} pewność identyfikacji: {}".format(
                    total_tax.value, total_tax.confidence
                )
            )


In [12]:
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
analyze_invoice()

--------Rozpoznana faktura #1--------
Nazwa dostawcy       : CONTOSO LTD.
Adres dostawcy       : AddressValue(house_number=123, po_box=None, road=456th St, city=New York, state=NY, postal_code=10001, country_region=None, street_address=123 456th St)
Nazwa klienta        : MICROSOFT CORPORATION


In [13]:
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
analyze_invoice_with_confidence()

--------Rozpoznana faktura #1--------
Nazwa dostawcy       : CONTOSO LTD. pewność identyfikacji: 0.917
Adres dostawcy       : AddressValue(house_number=123, po_box=None, road=456th St, city=New York, state=NY, postal_code=10001, country_region=None, street_address=123 456th St) pewność identyfikacji: 0.887
Nazwa klienta        : MICROSOFT CORPORATION pewność identyfikacji: 0.893
Identyfikator faktury: INV-100 pewność identyfikacji: 0.971
Data faktury         : 2019-11-15 pewność identyfikacji: 0.97
Suma                 : $110.0 pewność identyfikacji: 0.969
Termin płatnosci     : 2019-12-15 pewność identyfikacji: 0.969
Zamówienie           : PO-3333 pewność identyfikacji:0.943
Adres wysyłki        : AddressValue(house_number=123, po_box=None, road=Ship St, city=Redmond, state=WA, postal_code=98052, country_region=None, street_address=123 Ship St) pewność identyfikacji:0.888
-----------------------------------------------
Pozycje na fakturze  :
...Pozycja #1
......Opis        : Test for 